In [1]:
import requests
r = requests.get("https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=otc", verify=False) # get nonWEBAPI data
print(r.status_code)
html_str = r.text
print(len(html_str))
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_str)

C:\Users\richi\Anaconda3\envs\crawler\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200
38477


In [39]:
name = []
for i in soup.select(".name a"):
    d = i.text.strip()
    name.append(d)
ticker = []
Sname = []
for i,a in enumerate(name):
    t = name[i][:4]
    s = name[i][5:]
    ticker.append(t)
    Sname.append(s)
price = []
for i in soup.select(".name+ td"):
    d = i.text.strip()
    price.append(d)
volume = []
for i in soup.select("td:nth-child(9)"):
    d = i.text.strip()
    volume.append(d)
mkt_value = []
for i in soup.select("td:nth-child(10)"):
    d = i.text.strip()
    mkt_value.append(d)
import pandas as pd
Stock = pd.DataFrame()
Stock['Ticker'] = ticker
Stock['Name'] = Sname
Stock['Price'] = price
Stock['Volume'] = volume
Stock['Mkt_Value'] = mkt_value
print(Stock)

   Ticker   Name   Price Volume Mkt_Value
0    5274     信驊  640.00     79    0.5048
1    1565     精華  522.00     27    0.1408
2    4966  譜瑞-KY  508.00     27    0.1377
3    8436     大江  485.00    146    0.7094
4    6510     精測  433.00     21    0.0912
5    5904     寶雅  375.00      2    0.0075
6    3529     力旺  350.00     97    0.3404
7    6488    環球晶  332.00    720    2.3950
8    6643    M31  302.00     13    0.0392
9    8299     群聯  277.50    143    0.3937
10   3293     鈊象  270.50    642    1.7374
11   3081     聯亞  268.50    390    1.0444
12   2729     瓦城  258.00      1    0.0026
13   8437  大地-KY  258.00      7    0.0180
14   6121     新普  255.50    202    0.5190
15   3611     鼎翰  252.00     33    0.0834
16   6462     神盾  250.50  2,275    5.6627
17   3152     璟德  246.00     38    0.0942
18   5278     尚凡  226.50     43    0.0974
19   1264     德麥  215.00      3    0.0065
20   6561     是方  204.50    105    0.2151
21   5263     智崴  203.50    196    0.4013
22   8406  金可-KY  202.00     13   

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_stock_df(url):
    r = requests.get(url)
    html_str = r.text
    soup = BeautifulSoup(html_str)
    prices = [float(i.text) for i in soup.select(".name+ td")]
    volumes = [int(i.text.replace(",", "")) for i in soup.select("td:nth-child(9)")]
    mkt_values = [float(i.text)*100000000 for i in soup.select("td:nth-child(10)")]
    stock_df = pd.DataFrame()
    stock_df["ticker"] = tickers
    stock_df["company"] = companies
    stock_df["price"] = prices
    stock_df["volume"] = volumes
    stock_df["mkt_value"] = mkt_values
    return stock_df

tse_url = "https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=tse&n=100"
otc_url = "https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=otc&n=100"
tse = get_stock_df(tse_url)
tse["type"] = "上市"
otc = get_stock_df(otc_url)
otc["type"] = "上櫃"
df = tse.append(otc)
df = df.reset_index(drop=True)
df.head()
# ...
# pd.DataFrame: ticker(3008)/company(大立光)/price/volume/mkt_value/type
# 200 x 6

## Selenium

- `selenium` package
- Update Chrome / Firefox
- Download ChromeDriver / Geckodriver
    - ChromeDriver: <https://chromedriver.storage.googleapis.com/index.html?path=74.0.3729.6/>
    - Geckodriver: <https://github.com/mozilla/geckodriver/releases/tag/v0.24.0>

In [41]:
# test
from  selenium import webdriver

driver = webdriver.Firefox(executable_path = "geckodriver.exe")

In [42]:
driver.get("http://www.bbc.com/news")

In [43]:
driver.get("http://www.linkedin.com/in/chaoannricardo")

In [44]:
driver.back()

In [45]:
driver.forward()

In [46]:
driver.close()

### X Path Locator

In [59]:
from selenium import webdriver
driver = webdriver.Firefox(executable_path = "geckodriver.exe")
driver.get("https://www.imdb.com/")

# Select Search Bar
elem = driver.find_element_by_xpath("//input[@id='navbar-query']")
elem.send_keys("Avenger: Endgame (2019)")

# Press Search Button
elem = driver.find_element_by_xpath("//div[@class='magnifyingglass navbarSprite']")
elem.click()

# Filter "Movie"
elem = driver.find_element_by_xpath("//a[@href='/find?q=Avenger%3A%20Endgame%20%282019%29&s=tt&ttype=ft&ref_=fn_ft']")
elem.click()

# Enter Movie info Page
elem = driver.find_element_by_xpath("//a[contains(text(),'Avengers: Endgame')]")
elem.click()

In [51]:
driver.close()

### Exercise

In [90]:
# rating, genre, cast, countries, release_dates


imdb_homepage = "https://www.imdb.com/"

def get_movie_info(movie_titles):
    rating = []
    genre = []
    cast = []
    countries = []
    release_dates = []
    movie_info = {}
    from selenium import webdriver
    driver = webdriver.Firefox(executable_path = "geckodriver.exe")
    for i,a in enumerate(movie_titles):
        # Enter the Information Page of the movies 
        driver.get("https://www.imdb.com/")
        # Select Search Bar
        elem = driver.find_element_by_xpath("//input[@id='navbar-query']")
        elem.send_keys(a)
        # Press Search Button
        elem = driver.find_element_by_xpath("//div[@class='magnifyingglass navbarSprite']")
        elem.click()
        # Filter "Movie"
        elem = driver.find_element_by_xpath("//ul[@class='findTitleSubfilterList']/li[1]/a")
        elem.click()
        # Enter Movie info Page
        elem = driver.find_element_by_xpath("//div[@class='findSection'][1]/table[@class='findList']/tbody/tr[@class='findResult odd'][1]/td[@class='result_text']/a")
        elem.click()
        # Extract Rating Value
        elem = driver.find_element_by_xpath("//span[@itemprop='ratingValue']")
        r = float(elem.text)
        rating.append(r)
        # Extract Genre Value
        elem = driver.find_elements_by_xpath("//div[@class='subtext']/a")
        g = [i.text for i in elem]
        genre.append(g)
        # Extract Cast Value                                  
        elem = driver.find_elements_by_xpath("//td[2]/a")
        c = [i.text for i in elem]
        cast.append(c)
        # Extract Cast Value
        elem = driver.find_element_by_xpath("//div[@class='subtext']/a[4]")
        elem.click()
        elem = driver.find_elements_by_xpath("//tr[@class='ipl-zebra-list__item release-date-item']/td[@class='release-date-item__country-name']/a")
        co = [i.text for i in elem]
        countries.append(co)
        # Extract Released Date Value
        elem = driver.find_elements_by_xpath("//tr[@class='ipl-zebra-list__item release-date-item']/td[@class='release-date-item__date']")
        re = [i.text for i in elem]
        release_dates.append(re)
        movie = {
            "rating": rating,
            "genre": genre,
            "cast": cast,
            "releaseCountries": countries,
            "releaseDates": release_dates
        }
        movie_info[a] = movie
    driver.close()
    return movie_info

   
        
    '''
    import pandas as pd
    movie_df = pd.DataFrame()
    movie_df["movie_titles"] = movie_titles
    movie_df["rating"] = rating
    movie_df["cast"] = cast
    movie_df["countries"] = countries
    movie_df["release_dates"] = release_dates
    return movie_df
    ''' 

movie_titles = ["The Avengers (2012)", "Avengers: Age of Ultron (2015)", "Avengers: Infinity War (2018)", "Avengers: Endgame (2019)"]
        
results = get_movie_info(movie_titles)



In [92]:
import json
with open("movie.json", "w")as f:
    json.dump(results, f)

In [140]:
# 馬蜂窩 Example
from selenium import webdriver
driver = webdriver.Firefox(executable_path = "geckodriver.exe")
driver.get("http://www.mafengwo.cn/jd/10769/gonglve.html")
elem = driver.find_element_by_css_selector(".pg-next")
elem.click()

### Get With Cookies

In [105]:
cookie = {
    "over18":"1"
}
# Requests
import requests
r = requests.get("https://www.ptt.cc/bbs/Gossiping/index.html", verify=False, cookies=cookie) # get nonWEBAPI data
print(r.status_code)
html_str = r.text
print(html_str)

C:\Users\richi\Anaconda3\envs\crawler\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


200
<!DOCTYPE html>
<html>
	<head>
		<meta charset="utf-8">
		

<meta name="viewport" content="width=device-width, initial-scale=1">

<title>看板 Gossiping 文章列表 - 批踢踢實業坊</title>

<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-common.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-base.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-custom.css">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/pushstream.css" media="screen">
<link rel="stylesheet" type="text/css" href="//images.ptt.cc/bbs/v2.25/bbs-print.css" media="print">




	</head>
    <body>
		
<div id="topbar-container">
	<div id="topbar" class="bbs-content">
		<a id="logo" href="/bbs/">批踢踢實業坊</a>
		<span>&rsaquo;</span>
		<a class="board" href="/bbs/Gossiping/index.html"><span class="board-label">看板 </span>Gossiping</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href=

## Exercise: Fantasy Movies Information

In [137]:
cookies = {"COOKIE_LANGUAGE":"en"}
# Requests
# New
import requests
r = requests.get("http://www.fantasy-sky.com/ContentList.aspx?section=002", verify=False, cookies=cookies) # get nonWEBAPI data
html_str = r.text
# BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_str)
name = []
for i in soup.select(".movies-name"):
    d = i.text.strip()
    name.append(d)
# Family
r = requests.get("http://www.fantasy-sky.com/ContentList.aspx?section=002&category=00202", verify=False, cookies=cookies) # get nonWEBAPI data
html_str = r.text
# BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_str)
for i in soup.select(".movies-name"):
    d = i.text.strip()
    name.append(d)
# International
r = requests.get("http://www.fantasy-sky.com/ContentList.aspx?section=002&category=00203", verify=False, cookies=cookies) # get nonWEBAPI data
html_str = r.text
# BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_str)
for i in soup.select(".movies-name"):
    d = i.text.strip()
    name.append(d)
# Favourites
r = requests.get("http://www.fantasy-sky.com/ContentList.aspx?section=002&category=00204", verify=False, cookies=cookies) # get nonWEBAPI data
html_str = r.text
# BeautifulSoup
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_str)
for i in soup.select(".movies-name"):
    d = i.text.strip()
    name.append(d)

print(name)
print(len(name))

["A Dog's Way Home", 'The Kid Who Would Be King', 'Mary Poppins Returns', 'Aquaman', 'Glass', 'The Mule', 'The Upside', 'Destroyer', 'Spider-Man™: Into the Spider-Verse', 'Vice', 'Creed II', 'Ben Is Back', 'Robin Hood', 'Mortal Engines', 'The Scoundrels', 'Last Letter', 'A Land Imagined', 'Dying To Survive', 'Project Gutenberg', 'Hidden Man', 'High Flash', 'To My 19 Year-Old', 'How to Train Our Dragon', 'The House Where The Mermaid Sleeps', 'Million Dollar Man', 'Hichki', 'Territory Of Love', 'Rampant', 'Unstoppable', 'Intimate Strangers', 'Newton', 'Dogman', 'Hibiki', 'Code Blue: The Movie', "Samurai's Promise", 'The Peanuts Movie', 'Ferdinand', 'Where the Wild Things Are', 'Hop', 'Pan', 'Sing', 'Puss In Boots', 'The Lego Batman Movie', 'Wreck-It Ralph', 'The Polar Express', 'Trolls', 'Turbo', 'Yogi Bear', 'Rouge', 'A Better Tomorrow', 'A Chinese Ghost Story', 'Animal World', 'Fengshui', 'Believer', 'Monstrum', 'The Great Battle', 'The Negotiation', 'Euforia', 'Goodbye Berlin', 'Forev

In [138]:
# Get_Movie_Info Function
def get_movie_info1(movie_titles):
    rating = []
    time = []
    movie_info = {}
    from selenium import webdriver
    driver = webdriver.Firefox(executable_path = "geckodriver.exe")
    for i,a in enumerate(movie_titles):
        print(i,a)
        # Enter the Information Page of the movies 
        driver.get("https://www.imdb.com/")
        # Select Search Bar
        elem = driver.find_element_by_xpath("//input[@id='navbar-query']")
        elem.send_keys(a)
        # Press Search Button
        elem = driver.find_element_by_xpath("//div[@class='magnifyingglass navbarSprite']")
        elem.click()
        # Filter "Movie"
        elem = driver.find_element_by_xpath("//ul[@class='findTitleSubfilterList']/li[1]/a")
        elem.click()
        # Enter Movie info Page
        elem = driver.find_element_by_xpath("//div[@class='findSection'][1]/table[@class='findList']/tbody/tr[@class='findResult odd'][1]/td[@class='result_text']/a")
        elem.click()
        # Extract Rating Value
        try:
            elem = driver.find_element_by_xpath("//span[@itemprop='ratingValue']")
            r = float(elem.text)
            rating.append(r)
        except:
            rating.append("No Rating Value?")
        # Extract Time
        try:
            elem = driver.find_elements_by_xpath("//time")
            re = elem[1].text
            time.append(re)
        except:
            rating.append("No Movie Time?")
        movie = {
            "rating": rating,
            "time": time
        }
        movie_info[a] = movie
    driver.close()
    return movie_info


In [ ]:
get_movie_info1(name)
print(movie_info)